## Question 1

    Install Pipenv
    What's the version of pipenv you installed?
    Use --version to find out


In [1]:
#!pipenv install pipenv

In [2]:
!pipenv --version

pipenv, version 2022.10.4


## Question 2

    Use Pipenv to install Scikit-Learn version 1.0.2
    What's the first hash for scikit-learn you get in Pipfile.lock?

Note: you should create an empty folder for homework and do it there.

In [3]:
#!pipenv install scikit-learn==1.0.2

In [4]:
pipfile_lock = !cat Pipfile.lock

In [5]:
index = str(pipfile_lock).find('scikit-learn')
index

49545

In [6]:
str(pipfile_lock)[index:index+150]

'scikit-learn": {\', \'            "hashes": [\', \'                "sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b",\', \'         '

## Model
We've prepared a dictionary vectorizer and a model.

In [7]:
PREFIX='https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework'

In [8]:
#!wget $PREFIX/model1.bin
#!wget $PREFIX/dv.bin

## Question 3
Let's use these models!

    Write a script for loading these models with pickle
    Score this client:

{"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}

What's the probability that this client will get a credit card?

    0.162
    0.391
    0.601
    0.993


In [9]:
import pickle

In [10]:
with open('dv.bin', 'rb') as f_dv: 
    dv = pickle.load(f_dv)

with open('model1.bin', 'rb') as f_model: 
    model = pickle.load(f_model)

In [11]:
def predict(customer, dv, model):
    X = dv.transform([customer])
    y_pred = model.predict_proba(X)[0, 1]
    churn = y_pred >= 0.5
    result = {
        'churn_probability': round(float(y_pred),3),
        'churn': bool(churn)
    }
    return result

In [12]:
customer = {
    "reports": 0,
    "share": 0.001694,
    "expenditure": 0.12,
    "owner": "yes"
}

In [13]:
predict(customer, dv, model)

{'churn_probability': 0.162, 'churn': False}

## Question 4

Now let's serve this model as a web service

    Install Flask and gunicorn (or waitress, if you're on Windows)
    Write Flask code for serving the model
    Now score this client using requests:

url = "YOUR_URL"

client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}

requests.post(url, json=client).json()

What's the probability that this client will get a credit card?

    0.274
    0.484
    0.698
    0.928


In [14]:
!cat predict.py

import pickle

from flask import Flask
from flask import request
from flask import jsonify


with open('dv.bin', 'rb') as f_dv: 
    dv = pickle.load(f_dv)
with open('model1.bin', 'rb') as f_model: 
    model = pickle.load(f_model)

app = Flask('credit_card')

@app.route('/predict', methods=['POST'])
def predict():
    customer = request.get_json()

    X = dv.transform([customer])
    y_pred = model.predict_proba(X)[0, 1]
    churn = y_pred >= 0.5

    result = {
        'churn_probability': round(float(y_pred),3),
        'churn': bool(churn)
    }

    return jsonify(result)


if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0', port=9696)

In [15]:
!cat predict_response.py

import requests


url = 'http://localhost:9696/predict'

client = {
    "reports": 0,
    "share": 0.245,
    "expenditure": 3.438,
    "owner": "yes"
}

response = requests.post(url, json=client).json()
print(response)

In [16]:
!python predict_response.py

{'churn': True, 'churn_probability': 0.928}


## Docker

Install Docker. We will use it for the next two questions.

For these questions, we prepared a base image: svizor/zoomcamp-model:3.9.12-slim. You'll need to use it (see Question 5 for an example).

This image is based on python:3.9.12-slim and has a logistic regression model (a different one) as well a dictionary vectorizer inside.

https://hub.docker.com/r/svizor/zoomcamp-model

In [17]:
#!sudo docker pull svizor/zoomcamp-model:3.9.12-slim

## Question 5

Download the base image svizor/zoomcamp-model:3.9.12-slim. You can easily make it by using docker pull command.

So what's the size of this base image?

-    15 Mb

-    125 Mb

-    275 Mb

-    415 Mb

You can get this information when running docker images - it'll be in the "SIZE" column.

In [18]:
import getpass
import os

In [19]:
password = getpass.getpass()
command = "sudo -S docker images"
os.system('echo %s | %s' % (password, command))

········
REPOSITORY              TAG           IMAGE ID       CREATED          SIZE
3.9.12-slim             latest        2e28d00baa09   33 minutes ago   726MB
svizor/zoomcamp-model   3.9.12-slim   571a6fdc554b   5 days ago       125MB


[sudo] senha para arcangis: 

0

## Dockerfile

Now create your own Dockerfile based on the image we prepared.


Now complete it:

    Install all the dependencies form the Pipenv file
    Copy your Flask script
    Run it with Gunicorn

After that, you can build your docker image.

In [20]:
!cat dockerfile

FROM svizor/zoomcamp-model:3.9.12-slim

RUN pip install pipenv
WORKDIR /app                                                                

COPY ["Pipfile", "Pipfile.lock", "./"]

RUN pipenv install --deploy --system

COPY ["*.py", "*.bin", "./"]

EXPOSE 9696

ENTRYPOINT ["gunicorn", "--bind", "0.0.0.0:9696", "predict:app"]

In [21]:
#!sudo docker build -t 3.9.12-slim .

## Question 6

Let's run your docker container!

After running it, score this client once again.

What's the probability that this client will get a credit card now?

-    0.289
-    0.502
-    0.769
-    0.972


In [22]:
#!sudo docker run -it -p 9696:9696 svizor/zoomcamp-model:3.9.12-slim

In [23]:
!python predict_response.py

{'churn': True, 'churn_probability': 0.928}
